In [1]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from datetime import datetime, timedelta 

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
conn = sqlite3.connect('news1.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS head;
    CREATE TABLE head(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        head TEXT NOT NULL,
        cdate TEXT NOT NULL,
        wdate TEXT NOT NULL,
        ref INTEGER NOT NULL,
        page INTEGER NOT NULL
    );
''')

cur.executescript('''
    DROP TABLE IF EXISTS history;
    CREATE TABLE history(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        seen TEXT NOT NULL,
        ref INTEGER NOT NULL
    );
''')

In [ ]:
seen = list()
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}

# 시간변수 만들기
start_date = datetime.strptime('20190817', '%Y%m%d') 
end_date = datetime.strptime('20200817', '%Y%m%d') 

str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

url = 'https://www.donga.com/news/List?p=1&prod=news&ymd=' #주소 합치는거 고치기
urls = list()
for i in str_date_list:
    u = url + i +'&m='
    urls.append(u)      

count = 0
    
while urls:
    try:
        count += 1
        seed = urls.pop(-1)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.content.decode('utf-8','replace'), 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 1000)
        ''', [seed])
        
        for _ in [_['href'] for _ in dom.select('#content > div.page > a')
                  if _.has_attr('href') or _.has_attr('right on')]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
        
        if dom.select('#list_body_id > div.list_content > dl > dt') != None:
            head = [_.text.strip() for _ in dom.select('#content > div > div.rightList > a > span.tit') ]
            wdate = [_.text.strip() for _ in dom.select('#content > div > div > a > span.date') ]
            cdate = str(datetime.now()).split('.')[0]
            page = re.search('p=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,1000)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)

In [3]:
start_date = datetime.strptime('20190817', '%Y%m%d') 
end_date = datetime.today()

str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

url = 'https://www.donga.com/news/List?p=1&prod=news&ymd=' #주소 합치는거 고치기
urls = list()
for i in str_date_list:
    u = url + i +'&m='
    urls.append(u)  

In [4]:
urls

['https://www.donga.com/news/List?p=1&prod=news&ymd=20190817&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190818&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190819&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190820&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190821&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190822&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190823&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190824&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190825&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190826&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190827&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190828&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190829&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190830&m=',
 'https://www.donga.com/news/List?p=1&prod=news&ymd=20190831&m=',
 'https://

In [5]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}
seed = 'https://www.donga.com/news/List?p=1&prod=news&ymd=20200816&m='
resp = download(seed, headers=headers)
dom = BeautifulSoup(resp.content.decode('utf-8','replace'), 'html.parser')

In [6]:
dom

<!DOCTYPE doctype html>

<html lang="ko">
<head>
<title>동아닷컴</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="대한민국을 대표하는 정론지 동아일보의 디지털 창(窓)으로서 사회의 변화와 다양성을 아우르는 깊이있고 정확한 뉴스를 제공하고 있습니다." name="description"/>
<meta content="동아닷컴" name="writer"/>
<link href="https://image.donga.com/donga2013/images/common/donga_icon.png" rel="image_src"/>
<link href="https://www.donga.com/news/List" rel="canonical"/>
<link href="https://image.donga.com/pc/2020/css/donga_sub.css?t=20200721" rel="stylesheet" type="text/css"/>
<link href="https://image.donga.com/donga/css.v.2.0/slick.css" rel="stylesheet" type="text/css">
<script src="https://image.donga.com/donga/js.v.1.0/jquery-latest.js" type="text/javascript"></script>
<script src="https://image.donga.com/donga/js.v.1.0/jquery-ui.js" type="text/javascript"></script>
<script src="https://image.donga.com/donga/js.v.1.0/jquery-cookie.js" type="text/javascript">

In [8]:
seen = []
for _ in [_['href'] for _ in dom.select('#content > div.page > a')
          if _.has_attr('href') or _.has_attr('right on')]:
    newUrls = urljoin(seed, _)
    if newUrls not in urls and newUrls not in seen:
        urls.append(newUrls)

In [36]:
[_['href'] for _ in dom.select('#content > div.page > a')
          if _.has_attr('href') or _.has_attr('right on')]

['?p=1&prod=news&ymd=20200816&m=',
 '?p=21&prod=news&ymd=20200816&m=',
 '?p=41&prod=news&ymd=20200816&m=',
 '?p=61&prod=news&ymd=20200816&m=',
 '?p=none&prod=news&ymd=20200816&m=']

In [ ]:
dom.select('#content > div.page > a')

In [ ]:
[_['href'] for _ in dom.select('#content > div.page > a')
                  if _.has_attr('href') or _.has_attr('right on')]

In [ ]:
[_.text.strip() for _ in dom.select('#content > div > div.rightList > a > span.tit') ]

In [ ]:
dom.select('#content > div > div.rightList > a > span.tit')
#content > div:nth-child(8) > div.rightList > a > span.tit

In [ ]:
dom.select('#content > div > div > a > span.date')
#content > div:nth-child(22) > div > a > span.date

In [ ]:
[_.text.strip() for _ in dom.select('#content > div > div > a > span.date') ]

In [ ]:
seed

In [ ]:
page = re.search('p=(\d+)', urlparse(seed).query).group(1)
page